# 02 - Labeling Intent

This notebook is where we create our **Golden Dataset** — a high-quality, manually labeled sample of customer tweets, annotated by intent.

---

### Intent Categories

| Intent Label        | Description                                      |
|---------------------|------------------------------------------------|
| `cancel_service`    | Customer wants to cancel account or switch plan|
| `billing_issue`     | Complaints or questions about billing           |
| `technical_issue`   | Problems with device, network, or service       |
| `account_help`      | Issues with account management or login         |
| `upgrade_request`   | Requests to upgrade device or plan               |
| `general_question`  | General questions or inquiries                    |
| `positive_feedback` | Praise or compliments                             |
| `complaint`         | Negative feedback not related to billing or tech|
| `other`             | Anything else or unclear                          |

In [ ]:
import pandas as pd

# Loads Cleaned Tweets data
df = pd.read_csz("data/processed/cleaned_tweets.)